<a href="https://colab.research.google.com/github/acgowda/marketflow/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 16.0 MB/s 
     |████████████████████████████████| 63 kB 850 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
import os
import pickle
import datetime as dt
import yfinance as yf
import sqlite3
import datetime as dt

from plotly import express as px
import plotly.io as pio
import matplotlib as mpl

pio.templates.default = "plotly_white"
mpl.style.use('seaborn')
mpl.rcParams['figure.dpi'] = 120

In [ ]:
target = 'TSLA'
stock = yf.Ticker(target)
df = stock.history(start = '2022-1-1',end = '2022-4-27')
df.drop(['Dividends','Stock Splits'],axis = 1,inplace = True)

In [ ]:
df.head(10)

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03,1147.750000,1201.069946,1136.040039,1199.780029,34643800
2022-01-04,1189.550049,1208.000000,1123.050049,1149.589966,33416100
2022-01-05,1146.650024,1170.339966,1081.010010,1088.119995,26706600
2022-01-06,1077.000000,1088.000000,1020.500000,1064.699951,30112200
2022-01-07,1080.369995,1080.930054,1010.000000,1026.959961,28054900
2022-01-10,1000.000000,1059.099976,980.000000,1058.119995,30605000
2022-01-11,1053.670044,1075.849976,1038.819946,1064.400024,22021100
2022-01-12,1078.849976,1114.839966,1072.589966,1106.219971,27913000
2022-01-13,1109.069946,1115.599976,1026.540039,1031.560059,32403300


In [ ]:
stock.financials

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,2593000000.0,1491000000.0,1343000000.0,1460000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,6343000000.0,1154000000.0,-665000000.0,-1005000000.0
Minority Interest,1394000000.0,1454000000.0,1492000000.0,1390000000.0
Net Income,5519000000.0,721000000.0,-862000000.0,-976000000.0
Selling General Administrative,4517000000.0,3188000000.0,2646000000.0,2835000000.0
Gross Profit,13606000000.0,6630000000.0,4069000000.0,4042000000.0
Ebit,6523000000.0,1951000000.0,80000000.0,-253000000.0
Operating Income,6523000000.0,1951000000.0,80000000.0,-253000000.0
Other Operating Expenses,-27000000.0,-27000000.0,-27000000.0,-27000000.0


In [ ]:
# https://medium.com/automated-trading/obtain-40-technical-indicators-for-a-stock-using-python-247b32e85f30
# https://github.com/bukosabino/ta

In [ ]:
!pip install ta

  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=aecb4e9a29fcdee64aa9594743a4671ad79a7d37a32ab7d7313e19bb55566c1f
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a1fa7fb012667e55cb0643cfa65996bddecc
Successfully built ta


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ta.trend import MACD
from ta.momentum import StochasticOscillator

# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

# add moving averages to df
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA5_pct'] = df['Close'].rolling(window=5).mean().pct_change()

# MACD
macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.7,0.1,0.2])

# Plot OHLC on 1st subplot (using the codes from before)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             showlegend=False))
# add moving average traces
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))

# Plot volume trace on 2nd row 
colors = ['green' if row['Close'] - row['Open'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors
                    ), row=2, col=1)

# Plot MACD trace on 3rd row
colors = ['green' if val >= 0 
          else 'red' for val in macd.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     marker_color=colors,
                     name = 'Difference'
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='orange', width=2),
                         name = 'MACD'
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1),
                         name = 'Signal Line'
                        ), row=3, col=1)

fig.update_layout(height=800, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])

# update y-axis label
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", row=3, col=1)

fig.show()

data needed
- target stock (OLHCV), SP500, DJIA30, VIX, HSI index, CAC, FTSE, SZSE, Gold, Silver, rate of inflation, 10yr-2yr treasury spread,  

technical indicators
- 5 day EMA, 20 day EMA, VWAP, ichimoku cloud

data transformation
1. get data
2. compute technical indicators (MA, MACD, RSI, etc.)
3. normalise datasets
4. merge to main dataset

# Getting DATA

In [4]:
# get most recent list of SP500 tickers
def get_sp500_tickers():
  """
  Returns a data frame of the S&P 500 tickers from the 
  Wikipedia on the S&P 500 Index

  Also saves a pickle file of the tickers for future use
  """
  tickers = pd.read_html(
      'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']

  with open('sp500_tickers','wb') as f:
      pickle.dump(tickers,f)  
  return tickers 

sp500_tickers = get_sp500_tickers()

In [5]:
def compile_data(tickers,start,end,indices):
  """
  Will merge data from list of tickers and indices
  """
  main_df = pd.DataFrame() # our final dataframe
  count = 0 # keep track of progress

  for ticker in tickers:
    try:
      ticker = ticker.replace('.', '-')
      df = yf.Ticker(ticker).history(start = start,end = end)
      df.drop(['Dividends','Stock Splits'],axis = 1,inplace = True)

      # lower case the columns
      for col in df.columns:
        df.rename(columns = {col:col.lower()},inplace = True)
        #df.rename(columns = {col:f'{ticker}_{col}'},inplace = True)
        #col_names.append(f'{ticker}_{col}')

      # normalise values with pct_change, drop NAs
      df = df.pct_change().fillna(method = 'ffill')
      df.dropna(inplace = True)

      # replace infinities with max/min vals
      for col in df.columns:
        p_ind = df[df[col]==np.inf].index
        n_ind = df[df[col]==-np.inf].index

        if len(p_ind) > 0:
          df[col].replace(np.inf,max(df[col].drop(p_ind,axis = 0)))
        if len(n_ind) > 0:
          df[col].replace(-np.inf,min(df[col].drop(n_ind,axis = 0)))
      
      # get day of week; 0 = Monday,...,so on so forth
      df['day'] = list(pd.Series(df.index).apply(lambda x: str(x.weekday())))
      # get week of month
      #df['week'] = list(pd.Series(df.index).apply(lambda x: str(x.isocalendar()[1] - x.replace(day=1).isocalendar()[1] + 1)))
      # month of year
      df['month'] = list(pd.Series(df.index).apply(lambda x: str(x.month)))
      
      df = df.dropna()
      df = pd.get_dummies(df)

      # rolling window stuff
      df['ma5'] = df['close'].rolling(window=5).mean()
      df['ma20'] = df['close'].rolling(window=20).mean()
      df['ma60'] = df['close'].rolling(window=60).mean()
      df['ma200'] = df['close'].rolling(window=200).mean()

      for ind in indices:
        i_df = yf.Ticker(ind).history(start = start,end = end)
        try:
          i_df.drop(['Dividends','Stock Splits','Open','Low','High'],axis = 1,inplace = True)
        except:
          i_df.drop(['Open','Low','High'],axis = 1,inplace = True)

        for col in i_df.columns:
          i_df.rename(columns = {col:f'{ind}-{col.lower()}'},inplace = True)
        
        # normalise values with pct_change, drop NAs
        i_df = i_df.pct_change().fillna(method = 'ffill')
        i_df.dropna(inplace = True)

        # replace infinities with max/min vals
        for col in i_df.columns:
          p_ind = i_df[i_df[col]==np.inf].index
          n_ind = i_df[i_df[col]==-np.inf].index

          if len(p_ind) > 0:
            i_df[col].replace(np.inf,max(i_df[col].drop(p_ind,axis = 0)))
          if len(n_ind) > 0:
            i_df[col].replace(-np.inf,min(i_df[col].drop(n_ind,axis = 0)))
        

        #print(i_df.columns)
        i_df[f'{ind}-ma5'] = i_df[f'{ind}-close'].rolling(window=5).mean()
        i_df[f'{ind}-ma20'] = i_df[f'{ind}-close'].rolling(window=20).mean()
        i_df[f'{ind}-ma60'] = i_df[f'{ind}-close'].rolling(window=60).mean()
        i_df[f'{ind}-ma200'] = i_df[f'{ind}-close'].rolling(window=200).mean()
        df = pd.concat([df,i_df], axis=1, ignore_index=False)

      # remove NaNs 
      df.dropna(inplace = True)

      # add data to our final df
      if main_df.empty:
        main_df = df
      else:
        main_df = pd.concat([main_df,df],axis = 0)

      # progress counting
      count +=1 # increment for every stock addded
      if count % 50 == 0: # will let us know progress for every 50 stocks added
          print(f'Progress: {count}/{len(tickers)}')
    except:
      continue

  # drop any NaNs
  main_df = main_df.dropna(axis = 0)
  return main_df

In [ ]:
# nasdaq,dji,sp500,hsi,ftse,cac,gold,oil,10yr treasury
indices = ['^IXIC','^DJI','^GSPC','^HSI','^FTSE','^FCHI','GC=F','CL=F']

# get ~5 years of data for every stock
df = compile_data(sp500_tickers,'2017-01-01','2022-05-12',indices)

# ignore everything below this

In [ ]:
def get_data1(tickers,start,end):
  """
  Collect information on the following: 
  - target stock OLHC + V (daily)
  - indices: S&P 500, DJIA 30, NASDAQ, CAC, FTSE, SZSE, HSI (daily)
  - commodities: gold, wheat, oil, soy beans (daily)
  - macroeconomic indicators: U.S. inflation, global inflation, 10-2 year treasury spread (monthly/quarterly)

  Then store it in a database
  """
  # create/open connection
  conn = sqlite3.connect('ticker-data.db')

  # store information in the db
  for ticker in tickers:
    df = yf.Ticker.history(start = start,end = end)
    df.to_sql(ticker,conn,if_exists = 'replace',index = False)

  # close connection
  conn.close()

In [5]:
def get_sp500_tickers():
  """
  Returns a data frame of the S&P 500 tickers from the 
  Wikipedia on the S&P 500 Index

  Also saves a pickle file of the tickers for future use
  """
  tickers = pd.read_html(
      'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']

  with open('sp500_tickers','wb') as f:
      pickle.dump(tickers,f)  
  return tickers 

tickers = get_sp500_tickers()

def get_data2(tickers,start,end):
  """
  Collect information on the following: 
  - target stock OLHC + V (daily)
  - indices: S&P 500, DJIA 30, NASDAQ, CAC, FTSE, SZSE, HSI (daily)
  - commodities: gold, wheat, oil, soy beans (daily)
  - macroeconomic indicators: U.S. inflation, global inflation, 10-2 year treasury spread (monthly/quarterly)

  Then store it in a directory called 'data'
  """
  # create a directory to store all of our data
  if not os.path.exists('data'):
        os.makedirs('data')

  # store information in the data folder
  for ticker in tickers:
    df = yf.Ticker.history(start = start,end = end)
    df.to_csv(f'data/{ticker}.csv')

In [6]:
for t in tickers:
  t.replace('.', '-')

# start and end dates
start = dt.datetime(2000,1,1) 
end = dt.datetime.now()

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
499     YUM
500    ZBRA
501     ZBH
502    ZION
503     ZTS
Name: Symbol, Length: 504, dtype: object


In [ ]:
def compute_technicals(df):
  """
  Takes in a data frame and returns that same data frame with
  technical indicator columns added to it

  CAUTION: MAKE SURE TO DEAL WITH ANY VALUES IN THIS LIST [-np.inf,np.inf,np.isna()]
    - for [-np.inf, np.inf], we can 
      (1) convert them to [min(df[col]),max(df[col])] for that specific column
      (2) or just df.replace([-np.inf,np.inf],np.nan)

    - for np.isna(), we can take the ffill() then drop approach
  """

  # (1) create and add technical indicators
  df['sma50'] = df['Close'].rolling(min_periods = 50)

  # (2) ensure there are no nan values or inf values
  # replace infinites with NaN
  # df.replace([np.inf, -np.inf], np.nan, inplace=True)
  # df = df.ffill() 
  # df = df.dropna(axis = 0) # just in case
  
  # replace infinities with max/min vals
  for col in df.columns:
    df[col].replace[]
    p_indices = df[df[col]==float('inf')].index # indices of positive infinity
    n_indices = df[df[col]==-float('inf')].index # indices of negative infinity

    # replace +inf with max val in specific column
    for i in p_indices:
      df.at[i,col] = max(df[col].drop(p_indices))

    # replace -inf with min val in specific column
    for i in n_indices:
      df.at[i,col] = min(df[col].drop(n_indices))

In [ ]:
from sklearn.preprocessing import StandardScalar
def prep_data(tickers):
  """
  (1) compute technical indicators
  (2) normalise data sets
  (3) merge data sets
  """
  main_df = pd.DataFrame()
  scaler = StandardScaler()
  for count, ticker in enumerate(tickers):
      df = pd.read_csv(f'data/{ticker}.csv')
      df.set_index('Date',inplace=True) # set date column as index

      df[df.columns] = scaler.fit_transform(df)

      # initialise main_df
      if main_df.empty:
          main_df = df
      # concatenate dataframes row-wise
      else:
          main_df = pd.concat([main_df,df],axis =0)
          
      if count % 100 == 0:
          print(count)
          
  main_df.to_csv('sp500_data.csv')

In [ ]:
def get_sp500_data(start,reload_sp500 = False):
    """
    Creates a directory called "stocks_dfs" and stores the price 
    data of all the stocks in the S&P 500 index in that folder
    """
    tickers = []
    if reload_sp500==True:
        tickers = get_sp500_tickers()
    else:
        with open('sp500_tickers','rb') as f:
            tickers = pickle.load(f)
    
    for i in range(len(tickers)):
        tickers[i] = tickers[i].replace('.', '-')
    
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start = start
    end = dt.datetime.now() # Get up to the most recent time 

    for ticker in tickers:
        if not os.path.exists(f'stock_dfs/{ticker}.csv'):
            df = web.DataReader(ticker,'yahoo',start,end)
            df.to_csv(f'stock_dfs/{ticker}.csv') 
        else:
            print(f'Already have {ticker}')

In [ ]:
# dropna() because we will have na in first row
# due to normalising everything into a percentage change
df_norm = df.pct_change().dropna()
